# <center>Exercise 5</center>
## <center>Segmentation II</center>

In this exercise we will be using some methods of feature extraction to aid in the segmentation using unsupervised machine learning, specifically the algorithm k-means. We will be using the image 'cormoran.jpg'.

In [1]:
#importamos librerías necesarias
import numpy as np
import cv2
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans

import texturefilters as tf

https://scikit-learn.org/stable/about.html#citing-scikit-learn 
    
citation

1. Visual analysis of the image

Read and display the image 'cormoran.jpg'. What are the three distinct objects in the image? What characteristic best differentiates them?

Turn the image into gray scale and display its histogram? Could you use thresholding to separate the different objects in the image?

2. RGB characteristics and k-means

In this section we will try to use the different color components of the RGB image as characteristics for segmentation.

- First split the channels of the image

- Turn each channel into a vector of size (nrows*ncols, 1) using the function [np.reshape](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html).

- Display in a 3d scatter plot all the data from the 3 vectors, can you see any distict group?

[Here](https://matplotlib.org/3.1.1/gallery/mplot3d/scatter3d.html) is the documentation on 3d scatter plot. 


The space we displayed in the las segment is where we will apply [k-means](https://team.inria.fr/steep/files/2015/03/tp51.pdf). Since we have three objects we will use k = 3, that means we will have 3 randomized centroids.

To implement this algorithm we will be using [Kmeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) from sklearn.cluster, it should be in the cell with the imported libraries. We will be using the method fit_predict which asigns a label to each pixel according to the cluster assigned to them.

Fisrt concatenate the three vectors of the different components, you should get a matriz of size (nrows* ncols, 3). We will be using two commands to make it easier, the code should look like this:

<code>kmeans = KMeans()
prediction = kmeans.fit_predict()
</code>

In KMeans leave every value as its default except the number of clusters. The input for fit_predict should be the matrix with the concatenated vectors.

Now we will display scatter plot from last section adding the centroids in their positions after applying the algorithm. To access the positions of the cluster centers use the method kmeans.cluster_centers, the output of this method is an array with the coordinates of each center. 

<code>First_cluster = kmeans.cluster_centers_[:, 0]
</code>

In the scatter plot, paint the dots representing the centers a different color and make them bigger so they are easier to spot.

To see the actual segmentation of the image we will use the method kmeans.labels_, this creates a vectors size (nrows* ncols, 1) with the number of the cluster assigned to each pixel in the image. To display it as an image in false color (as we did in the last exercise with the labeled image) we have to return this vector to the original shape of the image.

Now display this labeled image, no colormap necessary.